<a href="https://colab.research.google.com/github/Mr-Katakitchen/ML-NTUA/blob/main/lab1/MM2024_25_ex1_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Άσκηση ταξινόμησης

Για τον πειραματισμό μας  θα χρησιμοποιήσουμε ως dataset το [Pima-Indians-diabetes](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). Θα φορτώσουμε το dataset και θα το χωρίσουμε σε data και σε labels.

In [141]:
from IPython.display import Image
from IPython.display import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# inline plotting instead of popping out
%matplotlib inline


# e.g., plot_decision_regions()
import os, sys
module_path = os.path.abspath(os.path.join('.'))
sys.path.append(module_path)


# Load CSV using Pandas from URL

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)
print(data.shape)
data.info() #shamone

# separate array into input and output components
# X: Features Y: Labels

y = data['class']
X = data.drop('class', axis=1)

X.shape, y.shape

(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   preg    768 non-null    int64  
 1   plas    768 non-null    int64  
 2   pres    768 non-null    int64  
 3   skin    768 non-null    int64  
 4   test    768 non-null    int64  
 5   mass    768 non-null    float64
 6   pedi    768 non-null    float64
 7   age     768 non-null    int64  
 8   class   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


((768, 8), (768,))

1. Χωρίστε τα data kai ta labels σε training (80%) και σε testing set (20%).

In [142]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=34)
#I change the random_state from time to time

2.  Έπειτα χωρίστε το training set σε 70% και σε 30% όπου το 30% θα το χρησιμοποιήσουμε για validation.

In [143]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.30, random_state=34)

In [145]:
# Print the shapes of each split
print("Training set shape (features):", X_train.shape)
print("Training set shape (labels):", y_train.shape)
print("Validation set shape (features):", X_val.shape)
print("Validation set shape (labels):", y_val.shape)
print("Test set shape (features):", X_test.shape)
print("Test set shape (labels):", y_test.shape)

# Check the distribution of labels in each split to ensure stratification worked
print("\nLabel distribution in training set:", np.bincount(y_train))
print("Label distribution in validation set:", np.bincount(y_val))
print("Label distribution in test set:", np.bincount(y_test))

#Making sure seperation is done correctly

Training set shape (features): (429, 8)
Training set shape (labels): (429,)
Validation set shape (features): (185, 8)
Validation set shape (labels): (185,)
Test set shape (features): (154, 8)
Test set shape (labels): (154,)

Label distribution in training set: [278 151]
Label distribution in validation set: [117  68]
Label distribution in test set: [105  49]


3. Επιλέξτε το καλύτερο αποτέλεσμα και εφαρμόστε αυτό το μοντέλο στα δεδομένα ελέγχου (test set).

In [146]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Create a dictionary of models to train
models = {
    "Logistic Regression": LogisticRegression(max_iter=500), #there was a warning if I didn't specify the iterator
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "SVM": SVC(),
    "Gradient Boosting": GradientBoostingClassifier()
}

best_model = None
best_score = 0

#I guess this is hold out cross validation
for name, model in models.items():
    model.fit(X_train, y_train)
    val_score = accuracy_score(y_val, model.predict(X_val))
    print(f"{name} Validation Accuracy: {val_score}")

    if val_score > best_score:
        best_score = val_score
        best_model = model

print("\nBest Model:", best_model)


Logistic Regression Validation Accuracy: 0.7621621621621621
Random Forest Validation Accuracy: 0.7621621621621621
K-Nearest Neighbors Validation Accuracy: 0.7027027027027027
SVM Validation Accuracy: 0.7135135135135136
Gradient Boosting Validation Accuracy: 0.7189189189189189

Best Model: LogisticRegression(max_iter=500)


In [150]:
from sklearn.metrics import classification_report

# Logistic Regression came up with the best score most of the time (not always)
# Random Forest Classifier comes second, SVM third
# Might be because Logistic Regression seems to be optimal for data with a binary outcome

best_model_prediction = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, best_model_prediction)
print(f"Test Accuracy: {test_accuracy}")

print("\nClassification Report:")
print(classification_report(y_test, best_model_prediction))

print("Labels' prediction ",best_model_prediction)
print("Labels from test set:", y_test.values)

Test Accuracy: 0.8246753246753247

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       105
           1       0.79      0.61      0.69        49

    accuracy                           0.82       154
   macro avg       0.81      0.77      0.78       154
weighted avg       0.82      0.82      0.82       154

Labels' prediction  [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 0]
Labels from test set: [0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0
 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0

4. Eκπαιδεύσετε έναν  Gaussian Naive Bayes  classifier, χρησιμοποιώντας το αρχικό σύνολο δεδομένο και εφαρμόζοντας 5 fold cross validation. Τυπώστε τα αποτελέσματα.

In [151]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

# Initialize the classifier
gnb = GaussianNB()

# Train the classifier:
gnb_model = gnb.fit(X_train, y_train)

# Perform 5-fold cross-validation
cv_scores = cross_val_score(gnb, X_train, y_train, cv=5)

# Print cross-validation results
print("Cross-Validation Accuracy Scores:", cv_scores)
print("Mean Cross-Validation Accuracy:", cv_scores.mean())

Cross-Validation Accuracy Scores: [0.73255814 0.70930233 0.6627907  0.76744186 0.8       ]
Mean Cross-Validation Accuracy: 0.7344186046511627


6. Eφαρμόστε το μοντέλο που εκπαιδεύσατε στα δεδομένα ελέγχου (test set)

In [152]:
gnb_prediction = gnb_model.predict(X_test)

gnb_accuracy = accuracy_score(y_test, gnb_prediction)
print(f"Test Accuracy: {gnb_accuracy}")

print("\nClassification Report:")
print(classification_report(y_test, gnb_prediction))

print("Labels' prediction ",gnb_prediction)
print("Labels from test set:", y_test.values)

Test Accuracy: 0.8376623376623377

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       105
           1       0.77      0.69      0.73        49

    accuracy                           0.84       154
   macro avg       0.82      0.80      0.81       154
weighted avg       0.83      0.84      0.84       154

Labels' prediction  [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0
 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0
 1 1 0 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0
 0 0 0 0 0 0]
Labels from test set: [0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 1 1 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0
 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0

7. Εκτυπώστε τον confusion matrix και για τους δύο ταξινομητές που εκπαιδεύσατε και ελέγξατε.


In [154]:
from sklearn.metrics import confusion_matrix

# Confusion matrix for best model according to previous comparison
cnf_matrix_bestModel = confusion_matrix(y_test, best_model_prediction)

  # τυπώνουμε το confusion matrix
print("confusion matrix for best model from before")
print('   0   1')
print('0', cnf_matrix_bestModel[0])
print('1', cnf_matrix_bestModel[1], '\n')

# Confusion matrix of gnb
cnf_matrix_gnb = confusion_matrix(y_test, gnb_prediction)

  # τυπώνουμε το confusion matrix
print("confusion matrix for gnb")
print('   0   1')
print('0', cnf_matrix_gnb[0])
print('1', cnf_matrix_gnb[1])

#Not too many false positives, false negatives
#Sometimes gnb does better, sometimes not
#Results vary with each random_state during data grouping

confusion matrix for best model from before
   0   1
0 [97  8]
1 [19 30] 

confusion matrix for gnb
   0   1
0 [95 10]
1 [15 34]
